In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import sympy as sp
from sympy.solvers import solve
import pandas as pd

## Модельная задача

### Аналитическое решение

In [2]:
k = 1. + np.cos(0.5) * np.cos(0.5)
q = 1.
f = np.sin(0.5) * np.sin(0.5)
beta1 = 1.
eps1 = 0
beta2 = 1.
eps2 = -1.

In [3]:
def analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2):
    from sympy.abc import a,b
    lmb = lambda q,k: np.sqrt(q/k)
    u = lambda c1,c2,lmb,x,f,q: c1 * np.exp(lmb*x) + c2 * np.exp(-lmb*x) + f/q

    lmb1 = lmb(q,k)
    lmb2 = - lmb1
    equations = [(k*lmb1-beta1)*a+(k*lmb2-beta1)*b-beta1*f/q+eps1,
                 -(k*lmb1-beta2)*np.exp(lmb1)*a-(k*lmb2+beta2)*np.exp(lmb2)*b-beta2*f/q+eps2]
    constants = solve(equations,[a,b],dict=False)
    c1 = constants[a]
    c2 = constants[b]
    u1 = u(c1,c2,lmb1,x,f,q)
    return u1

In [4]:
def analytic_points(k,q,f,beta1,eps1,beta2,eps2):
        x1 = np.linspace(0,1,11)
        y1 = []
        y2 = []
        for x in x1:
                res1 = analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2)
                y1.append(res1)
        return y1

### Метод прогонки

In [5]:
def progonka_1(n,k,q,f,beta1,eps1,beta2,eps2):
    h = 1/n
    x1 = np.linspace(0,1,11)
    x0 = 1
    solutions = np.zeros(11)
    u = np.zeros(n+1)
    alpha = np.zeros(n+1)
    beta = np.zeros(n+1)
    al = k; bl = -2*k - q*h**2
    cl = k
    dl = -f*h**2
    a0 = k
    b0 = -k-beta1*h
    c0 = 0
    d0 = -eps1*h
    aL = 0
    bL = -k-beta2*h
    cL=k
    dL = -eps2*h
    
    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al/(bl+cl*alpha[i-1])
        beta[i] = (dl - cl*beta[i-1])/(bl+cl*alpha[i-1])

    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]
    count = 9
    t = 0.9
    
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [6]:
x = np.linspace(0,1,11)
u1 = progonka_1(10,k,q,f,beta1,eps1,beta2,eps2)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
delta1 = []; 
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
df = pd.DataFrame({"x": x, "u аналит.": a1, "u числ. ": u1 ,"delta u": delta1})
display(df)
print("Максимальная норма ошибки =", max(delta1))

,x,u аналит.,u числ.,delta u
0,0.0,-1.96386223050994,-0.173943,1.78991904696278
1,0.1,-2.08110922395959,-0.183770,1.89733958105845
2,0.2,-2.21141750994839,-0.195933,2.01548477960768
3,0.3,-2.35552357723608,-0.210501,2.14502241922991
4,0.4,-2.51424189819457,-0.227557,2.28668467182434
5,0.5,-2.68846953212034,-0.247197,2.44127224294761
6,0.6,-2.87919119531943,-0.269532,2.60965889757390
7,0.7,-3.08748482662067,-0.294688,2.79279639881799
8,0.8,-3.31452767977289,-0.322808,2.99171988753615
9,0.9,-3.56160297715952,-0.354049,3.20755373321168


Максимальная норма ошибки = 3.44151788919579


## Задача с переменными коеффициентами

In [7]:
def progonka_2(n,beta1,eps1,beta2,eps2):
    h = 1/n
    x1 = np.linspace(0,1,11); x0 = 1; x = 0 + h
    solutions = np.zeros(11)
    k = lambda x: x**2 + 1
    q = lambda x: x + 2
    f = lambda x: np.cos(x)
    
    u = np.zeros(n+1); alpha = np.zeros(n+1); beta = np.zeros(n+1)
    al = lambda x: k(x+h/2); bl = lambda x: -(k(x+h/2)+k(x-h/2)+q(x)*h**2); cl = lambda x: k(x-h/2); dl = lambda x: -f(x)*h**2
    a0 = k(0); b0 = -k(0)-beta1*h; c0 = 0; d0 = -eps1*h
    aL = 0; bL = -k(1)-beta2*h; cL=k(1); dL = -eps2*h

    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al(x)/(bl(x)+cl(x)*alpha[i-1])
        beta[i] = (dl(x) - cl(x)*beta[i-1])/(bl(x)+cl(x)*alpha[i-1])
        x += h
    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]; count = 9; t = 0.9
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [8]:
a2 = progonka_2(10**6,beta1,eps1,beta2,eps2)

In [9]:
x = np.linspace(0,1,11)
u2 = progonka_2(30,beta1,eps1,beta2,eps2)
delta2 = []
for i in range(len(u2)):
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u аналит.": a2, "u числ. ": u2 ,"delta u ": delta2})
display(df)
print("Максимальная норма ошибки =", max(delta2))

,x,u аналит.,u числ.,u числ.- u аналит.
0,0.0,0.068980,0.062804,0.006176
1,0.1,0.071598,0.066197,0.005401
2,0.2,0.065917,0.061164,0.004753
3,0.3,0.052600,0.048370,0.004230
4,0.4,0.032564,0.028744,0.003821
5,0.5,0.006852,0.003337,0.003515
6,0.6,-0.023495,-0.026796,0.003301
7,0.7,-0.057519,-0.060682,0.003164
8,0.8,-0.094387,-0.097481,0.003093
9,0.9,-0.133415,-0.136494,0.003079


Максимальная норма ошибки = 0.0061755943621878695


## Таблица результатов

In [10]:
x = np.linspace(0,1,11)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
a2 = progonka_2(10**6,beta1,eps1,beta2,eps2)

In [11]:
delta1 = []; delta2 = []; 
u1 = progonka_1(20,k,q,f,beta1,eps1,beta2,eps2)
u2 = progonka_2(20,beta1,eps1,beta2,eps2)
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u мод. аналит.": a1, "u мод. числ. ": u1 ,"du мод.": delta1, 
                  "u пер. аналит.": a2, "u пер. числ. ": u2 ,"du": delta2})
display(df)
print("Максимальная норма ошибки модельной задачи =", max(delta1))
print("Максимальная норма ошибки реальной задачи = ", max(delta2))

,x,u мод. аналит.,u мод. числ.,u мод. числ.- u мод. аналит.,u пер. аналит.,u пер. числ.,u пер. числ.- u пер. аналит.
0,0.0,-1.96386223050994,-0.165724,1.79813810410915,0.068980,0.059641,0.009339
1,0.1,-2.08110922395959,-0.175652,1.90545766976458,0.071598,0.063426,0.008172
2,0.2,-2.21141750994839,-0.187871,2.02354695204751,0.065917,0.058719,0.007198
3,0.3,-2.35552357723608,-0.202450,2.15307338732100,0.052600,0.046188,0.006412
4,0.4,-2.51424189819457,-0.219473,2.29476905510751,0.032564,0.026764,0.005800
5,0.5,-2.68846953212034,-0.239035,2.44943481578029,0.006852,0.001507,0.005345
6,0.6,-2.87919119531943,-0.261246,2.61794483700624,-0.023495,-0.028521,0.005026
7,0.7,-3.08748482662067,-0.286233,2.80125153452335,-0.057519,-0.062344,0.004825
8,0.8,-3.31452767977289,-0.314137,3.00039095517809,-0.094387,-0.099111,0.004724
9,0.9,-3.56160297715952,-0.345114,3.21648863264739,-0.133415,-0.138124,0.004709


Максимальная норма ошибки модельной задачи = 3.45076594894183
Максимальная норма ошибки реальной задачи =  0.009339288040224658
